In [12]:
import re
import pandas as pd
from pathlib import Path

FILE = Path("/Users/danarapp/Desktop/energypattern-keyword-search/processing_pipeline/analysis/analyzed_matches/second_iteration.xlsx")

SHEETS = [
    "git_datatransfer",
    "comments_datatransfer",
    "git_UI",
    "comments_UI",
    "git_code_optimization",
    "comments_code_optimization",
]

REVIEW_COLS = ["row_id", "matched_word", "sentence", "source", "url", "pattern", "commit_url", "comment"]

# --- Helpers ---------------------------------------------------------------

_hlx = re.compile(r'^=HYPERLINK\(\s*"([^"]+)"\s*,\s*"[^"]*"\s*\)$', flags=re.IGNORECASE)

def from_hyperlink(cell):
    """Extract raw URL from Excel HYPERLINK formula; otherwise return the original value."""
    if pd.isna(cell):
        return ""
    if isinstance(cell, str):
        m = _hlx.match(cell.strip())
        if m:
            return m.group(1)
        return cell.strip()
    return str(cell)

def ensure_schema(df: pd.DataFrame) -> pd.DataFrame:
    """Make sure required columns exist with sane defaults."""
    d = df.copy()
    for col in REVIEW_COLS:
        if col not in d.columns:
            d[col] = ""  # default empty if missing
    # row_id often comes back as float from Excel; coerce to Int64 if possible
    if "row_id" in d.columns:
        d["row_id"] = pd.to_numeric(d["row_id"], errors="coerce").astype("Int64")
    # strip hyperlink formulas to raw URLs
    for c in ("url", "commit_url"):
        if c in d.columns:
            d[c] = d[c].apply(from_hyperlink)
    # keep only expected columns in a stable order
    return d.reindex(columns=REVIEW_COLS)

# --- Loaders ---------------------------------------------------------------

def load_review_workbook(path: str | Path, sheets=SHEETS) -> dict[str, pd.DataFrame]:
    """
    Read specified sheets and clean each DataFrame.
    Returns a dict: {sheet_name: cleaned_df}
    """
    # dtype=str ensures we don’t lose leading zeros or get Excel’s auto-typing;
    # we’ll coerce specific columns ourselves if needed.
    raw = pd.read_excel(path, sheet_name=sheets, engine="openpyxl", dtype=str)
    cleaned = {}
    for name in sheets:
        df = raw[name]
        cleaned[name] = ensure_schema(df)
    return cleaned

def load_and_concat(path: str | Path, sheets=SHEETS) -> pd.DataFrame:
    """
    Load all sheets, clean, and concatenate into one DataFrame with a 'sheet' column.
    """
    parts = []
    for name, df in load_review_workbook(path, sheets).items():
        parts.append(df.assign(sheet=name))
    out = pd.concat(parts, ignore_index=True)
    return out



In [15]:
from pathlib import Path

# Paths
FILE_1 = Path("/Users/danarapp/Desktop/energypattern-keyword-search/processing_pipeline/analysis/analyzed_matches/second_iteration.xlsx")
FILE_2 = Path("/Users/danarapp/Desktop/energypattern-keyword-search/processing_pipeline/analysis/analyzed_matches/wagtail_pattern_verification.xlsx")

# Load both workbooks
df1 = load_and_concat(FILE_1)
df2 = load_and_concat(FILE_2)

# Combine them (ignore index to get a fresh continuous index)
df_all = pd.concat([df1, df2], ignore_index=True)

print(f"✅ Combined dataframe created with {len(df_all):,} total rows.")
print(f"📄 From file 1: {len(df1):,}, from file 2: {len(df2):,}")
print("🧩 Columns:", list(df_all.columns))
print("Sheets represented:", df_all['sheet'].unique())


ValueError: Worksheet named 'git_UI' not found

In [14]:
# Load and concatenate all sheets into a single DataFrame
df_all = load_and_concat(FILE)

# Robust analyzed/unanalyzed flags
is_unanalyzed = df_all["pattern"].isna() | df_all["pattern"].astype(str).str.strip().eq("")
is_analyzed   = ~is_unanalyzed

total = len(df_all)
n_an  = int(is_analyzed.sum())
n_un  = int(is_unanalyzed.sum())

print(f"✅ Total matches: {total:,}")
print(f"🔍 Analyzed matches: {n_an:,}")
print(f"🕳️ Unanalyzed matches: {n_un:,}")
print(f"📊 Percentage analyzed: {n_an/total*100:.1f}%")

# Breakdown per sheet
summary = (
    df_all.assign(analyzed=is_analyzed)
          .groupby("sheet")["analyzed"]
          .agg(analyzed="sum", total="count")
)
summary["percent_analyzed"] = 100 * summary["analyzed"] / summary["total"]
print("\nBreakdown by sheet:\n")
print(summary.sort_index())


✅ Total matches: 735
🔍 Analyzed matches: 150
🕳️ Unanalyzed matches: 585
📊 Percentage analyzed: 20.4%

Breakdown by sheet:

                            analyzed  total  percent_analyzed
sheet                                                        
comments_UI                        0      8          0.000000
comments_code_optimization         0     33          0.000000
comments_datatransfer             33     83         39.759036
git_UI                             0     28          0.000000
git_code_optimization              0    146          0.000000
git_datatransfer                 117    437         26.773455
